In [1]:
import numpy as np
import pandas as pd
import os.path, matplotlib
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('data/data.csv')

In [3]:
# # one hot encoding for linear regression
# encoded_labels = pd.get_dummies(data.label)
# data = pd.concat([data, encoded_labels],axis=1)
# data = data.drop(['label','filename'], axis=1)
# df = data.drop(['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock'], axis=1)
# x_train, x_test, y_train, y_test = train_test_split(df, encoded_labels, test_size=0.1)
# # linear model
# from sklearn import linear_model
# linreg = linear_model.LinearRegression()
# linreg_model = linreg.fit(x_train, y_train)
# linreg_pred = linreg_model.predict(x_test)
# linreg_model.score(x_test,y_test)

In [10]:
# no encoding
label = data.label
df = data.drop(['label','filename'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(df, label, test_size=0.1)

In [11]:
# linear kernel svc
from sklearn.svm import SVC
linsvc = SVC(kernel='linear')
linsvc_model = linsvc.fit(x_train, y_train)
linsvc_pred = linsvc_model.predict(x_test)
print(linsvc_model.score(x_test, y_test))

0.49


In [7]:
print(x_train.shape)
print(y_train.shape)

(900, 28)
(900,)
